# Perceiver Full MIDI (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (Setup Environment)

In [ ]:
#@title nvidia-smi gpu check
!nvidia-smi

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/Perceiver-Music-Transformer
!pip install einops
!pip install torch
!pip install torch-summary

!pip install tqdm
!pip install matplotlib

!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import random
import copy
import math
from collections import OrderedDict

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

import torch
from torchsummary import summary

print('Loading core modules...')
os.chdir('/content/Perceiver-Music-Transformer')

import TMIDIX

from perceiver_ar_pytorch import PerceiverAR
from autoregressive_wrapper import AutoregressiveWrapper

from midi2audio import FluidSynth
from IPython.display import Audio, display

os.chdir('/content/')
print('Done!')

# (DOWNLOAD MODEL)

In [ ]:
#@title Download Perceiver Pre-Trained Full MIDI Model
!wget --no-check-certificate -O 'Perceiver-Full-MIDI-Model.pth' "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118756&authkey=ANb1Z64PPHP2GxM"

# (LOAD)

In [ ]:
#@title Load/Reload the model

full_path_to_model_checkpoint = "/content/Perceiver-Full-MIDI-Model.pth" #@param {type:"string"}

print('Loading the model...')
# Load model

# constants

SEQ_LEN = 8160 * 6 # 49k
PREFIX_SEQ_LEN = (8160 * 6) - 1020

model = PerceiverAR(
    num_tokens = 132,
    dim = 1024,
    depth = 24,
    heads = 16,
    dim_head = 64,
    cross_attn_dropout = 0.5,
    max_seq_len = SEQ_LEN,
    cross_attn_seq_len = PREFIX_SEQ_LEN
)
model = AutoregressiveWrapper(model)
model.cuda()

state_dict = torch.load(full_path_to_model_checkpoint)

model.load_state_dict(state_dict)

model.eval()

print('Done!')

# Model stats
summary(model)

# (GENERATE)

# Continuation

In [ ]:
#@title Load Seed/Custom MIDI
full_path_to_custom_MIDI_file = "/content/Perceiver-Music-Transformer/Perceiver-MI-Seed-1.mid" #@param {type:"string"}

print('Loading custom MIDI file...')
score = TMIDIX.midi2ms_score(open(full_path_to_custom_MIDI_file, 'rb').read())

events_matrix = []

inputs = []

itrack = 1

#==================================================

# Memories augmentator

def augment(inputs):

  outs = []
  outy = []

  for i in range(1, 12):

    out1 = []
    out2 = []

    for j in range(0, len(inputs), 4):
      note = inputs[j:j+4]

      if (note[0] // 11) != 9:
        aug_note1 = copy.deepcopy(note)
        aug_note2 = copy.deepcopy(note)
        aug_note1[3] += i
        aug_note2[3] -= i
      else:
        aug_note1 = note
        aug_note2 = note

      out1.append(aug_note1)
      out2.append(aug_note2)

    outs.append(out1[random.randint(0, int(len(out1) / 2)):random.randint(int(len(out1) / 2), len(out1))])
    outs.append(out2[random.randint(0, int(len(out2) / 2)):random.randint(int(len(out2) / 2), len(out2))])

  for i in range(64):
    outy.extend(random.choice(outs))

  outy1 = []
  for o in outy:
    outy1.extend(o)

  return outy1

#==================================================

while itrack < len(score):
    for event in score[itrack]:         
      events_matrix.append(event)
    itrack += 1

# Sorting...
events_matrix.sort(key=lambda x: x[1])

# recalculating timings
for e in events_matrix:
    e[1] = int(e[1] / 10)
    if e[0] == 'note':
      e[2] = int(e[2] / 20)

# final processing...

data_tags = ['note', 
             'control_change', # 128
             'patch_change', # 129
             'pitch_wheel_change', # 130
            ] # SOS/EOS 131

melody_chords = []

pe = events_matrix[0]
for e in events_matrix:
  if e[0] in data_tags:

    if e[0] == 'note':
      # ['note', start_time, duration, channel, note, velocity]
      time = max(0, min(127, e[1]-pe[1]))
      duration = max(1, min(127, e[2]))
      channel = max(0, min(15, e[3]))
      pitch = max(1, min(127, e[4]))
      velocity = max(1, min(127, e[5]))

      melody_chords.extend([131, time, duration, channel, pitch, velocity])

    if e[0] == 'control_change':
      # ['control_change', dtime, channel, controller(0-127), value(0-127)]
      time = max(0, min(127, e[1]-pe[1]))
      channel = max(0, min(15, e[2]))
      controller = max(0, min(127, e[3]))
      value = max(0, min(127, e[4]))

      melody_chords.extend([131, 128, time, channel, controller, value])

    if e[0] == 'patch_change':
      # ['patch_change', dtime, channel, patch]
      time = max(0, min(127, e[1]-pe[1]))
      channel = max(0, min(15, e[2]))
      patch = max(0, min(127, e[3]))

      melody_chords.extend([131, 129, time, channel, patch, 0])

    if e[0] == 'pitch_wheel_change':
      # ['pitch_wheel_change', dtime, channel, pitch_wheel]
      time = max(0, min(127, e[1]-pe[1]))
      channel = max(0, min(15, e[2]))
      pitch_wheel = max(0, min(127, ((e[3] // 128)+64)))

      melody_chords.extend([131, 130, time, channel, pitch_wheel, 0])

    pe = e # Previous event

# Break between compositions
inputs.extend([131, 127, 127, 0, 0, 0])
inputs.extend(melody_chords)

# =================================

out1 = inputs

if len(out1) != 0:
    
    song = out1
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []

    song1 = []

    for s in song:
      if s < 131:
        son.append(s)

      else:
        if len(son) == 5:
          song1.append(son)
        son = []
        #son.append(s)
    
    for s in song1:

      if s[0] < 128: # note event
        
        time += s[0] * 10
        dur = s[1] * 20
        channel = s[2]
        pitch = s[3]
        vel = s[4]
                                  
        song_f.append(['note', time, dur, channel, pitch, vel ])

      if s[0] == 128: # control_change event
        time += s[1] * 10
        channel = s[2]
        controller = s[3]
        value = s[4]

        song_f.append(['control_change', time, channel, controller, value])

      if s[0] == 129: # patch_change event
        time += s[1] * 10
        channel = s[2]
        patch = s[3]

        song_f.append(['patch_change', time, channel, patch])

      if s[0] == 130: # pitch_wheel_change event
        time += s[1] * 10
        channel = s[2]
        pitch_wheel = (s[3] - 64) * 128

        song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Perceiver',  
                                                        output_file_name = '/content/Perceiver-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)
    print('Done!')

print('Displaying resulting composition...')
fname = '/content/Perceiver-Music-Composition'

x = []
y =[]
c = []

colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver', 'white', 'gold', 'silver', 'silver']

for s in song_f:
  if s[0] == 'note' and s[4] != 0:
    x.append(s[1] / 1000)
    y.append(s[4])
    c.append(colors[s[3]])

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
display(Audio(str(fname + '.wav'), rate=16000))

plt.figure(figsize=(14,5))
ax=plt.axes(title=fname)
ax.set_facecolor('black')

plt.scatter(x,y, c=c)
plt.xlabel("Time")
plt.ylabel("Pitch")
plt.show()

# Continuation

In [ ]:
#@title Single Continuation Block Generator

#@markdown NOTE: Play with the settings to get different results
number_of_prime_tokens = 510 #@param {type:"slider", min:120, max:510, step:6}
number_of_tokens_to_generate = 510 #@param {type:"slider", min:60, max:510, step:6}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

#===================================================================
print('=' * 70)
print('Perceiver Music Model Continuation Generator')
print('=' * 70)

print('Generation settings:')
print('=' * 70)
print('Number of prime tokens:', number_of_prime_tokens)
print('Number of tokens to generate:', number_of_tokens_to_generate)
print('Model temperature:', temperature)

print('=' * 70)
print('Generating...')

# inp = augment(inputs)

inp = inputs * math.ceil((8160 * 6) / len(inputs))

inp = inp[:(8160 * 6)]

inp = inp[510+len(inputs[:number_of_prime_tokens]):] + inputs[:number_of_prime_tokens]

inp1 = torch.LongTensor(inp).cuda()

out = model.generate(inp1[None, ...], 
                     number_of_tokens_to_generate, 
                     temperature=temperature)  

out1 = out.cpu().tolist()[0]

if len(out1) != 0:
    
    song = inputs[:number_of_prime_tokens] + out1
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []

    song1 = []

    for s in song:
      if s < 131:
        son.append(s)

      else:
        if len(son) == 5:
          song1.append(son)
        son = []
        #son.append(s)
    
    for s in song1:

      if s[0] < 128: # note event
        
        time += s[0] * 10
        dur = s[1] * 20
        channel = s[2]
        pitch = s[3]
        vel = s[4]
                                  
        song_f.append(['note', time, dur, channel, pitch, vel ])

      if s[0] == 128: # control_change event
        time += s[1] * 10
        channel = s[2]
        controller = s[3]
        value = s[4]

        song_f.append(['control_change', time, channel, controller, value])

      if s[0] == 129: # patch_change event
        time += s[1] * 10
        channel = s[2]
        patch = s[3]

        song_f.append(['patch_change', time, channel, patch])

      if s[0] == 130: # pitch_wheel_change event
        time += s[1] * 10
        channel = s[2]
        pitch_wheel = (s[3] - 64) * 128

        song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Perceiver',  
                                                        output_file_name = '/content/Perceiver-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)
    print('Done!')

print('Displaying resulting composition...')
fname = '/content/Perceiver-Music-Composition'

x = []
y =[]
c = []

colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver', 'white', 'gold', 'silver', 'silver']

for s in song_f:
  if s[0] == 'note' and s[4] != 0:
    x.append(s[1] / 1000)
    y.append(s[4])
    c.append(colors[s[3]])

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
display(Audio(str(fname + '.wav'), rate=16000))

plt.figure(figsize=(14,5))
ax=plt.axes(title=fname)
ax.set_facecolor('black')

plt.scatter(x,y, c=c)
plt.xlabel("Time")
plt.ylabel("Pitch")
plt.show()

In [ ]:
#@title Auto-Continue Custom MIDI

number_of_continuation_notes = 100 #@param {type:"slider", min:10, max:500, step:10}
number_of_prime_tokens = 510 #@param {type:"slider", min:120, max:510, step:6}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

#===================================================================
print('=' * 70)
print('Perceiver Music Model Auto-Continuation Generator')
print('=' * 70)

print('Generation settings:')
print('=' * 70)
print('Number of continuation notes:', number_of_continuation_notes)
print('Number of prime tokens:', number_of_prime_tokens)
print('Model temperature:', temperature)

print('=' * 70)
print('Generating...')

out2 = copy.deepcopy(inputs[:number_of_prime_tokens])

# aug_inp = augment(inputs)

for i in tqdm(range(number_of_continuation_notes)):

  # inp = copy.deepcopy(aug_inp)

  inp = inputs * math.ceil((8160 * 6) / len(inputs))

  inp = inp[:(8160 * 6)]

  inp = inp[510+len(out2):] + out2

  inp = torch.LongTensor(inp).cuda()

  out = model.generate(inp[None, ...], 
                      6, 
                      temperature=temperature)  

  out1 = out.cpu().tolist()[0]
  out2.extend(out1)

if len(out2) != 0:
    
    song = out2
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []

    song1 = []

    for s in song:
      if s < 131:
        son.append(s)

      else:
        if len(son) == 5:
          song1.append(son)
        son = []
        #son.append(s)
    
    for s in song1:

      if s[0] < 128: # note event
        
        time += s[0] * 10
        dur = s[1] * 20
        channel = s[2]
        pitch = s[3]
        vel = s[4]
                                  
        song_f.append(['note', time, dur, channel, pitch, vel ])

      if s[0] == 128: # control_change event
        time += s[1] * 10
        channel = s[2]
        controller = s[3]
        value = s[4]

        song_f.append(['control_change', time, channel, controller, value])

      if s[0] == 129: # patch_change event
        time += s[1] * 10
        channel = s[2]
        patch = s[3]

        song_f.append(['patch_change', time, channel, patch])

      if s[0] == 130: # pitch_wheel_change event
        time += s[1] * 10
        channel = s[2]
        pitch_wheel = (s[3] - 64) * 128

        song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Perceiver',  
                                                        output_file_name = '/content/Perceiver-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)
    print('Done!')

print('Displaying resulting composition...')
fname = '/content/Perceiver-Music-Composition'

x = []
y =[]
c = []

colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver', 'white', 'gold', 'silver', 'silver']

for s in song_f:
  if s[0] == 'note' and s[4] != 0:
    x.append(s[1] / 1000)
    y.append(s[4])
    c.append(colors[s[3]])

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
display(Audio(str(fname + '.wav'), rate=16000))

plt.figure(figsize=(14,5))
ax=plt.axes(title=fname)
ax.set_facecolor('black')

plt.scatter(x,y, c=c)
plt.xlabel("Time")
plt.ylabel("Pitch")
plt.show()

# Congrats! You did it! :)